## 3. <a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-CreateTableCreate/Drop/TruncateTable'>创建表</a>,<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-CreateTableCreate/Drop/TruncateTable'>加载数据</a>


#### 创建内部表表,创内部表,在删除表结构的时候,表数据也会被删除.
    create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
#### 加载文件    
    LOAD DATA LOCAL INPATH '/root/data.txt'  INTO TABLE psn;
#### 查看描述
    desc formated psn;

#### 创建外部表,创建的时候要加入LOCATION,HIVE只维护表结构,不会动表数据
     CREATE EXTERNAL TABLE page_view(
         viewTime INT, 
         userid BIGINT,
         page_url STRING, 
         referrer_url STRING,
         ip STRING COMMENT 'IP Address of the User',
         country STRING COMMENT 'country of origination'
         )
     COMMENT 'This is the staging page view table'
         ROW FORMAT DELIMITED FIELDS TERMINATED BY '\054'
         STORED AS TEXTFILE
     LOCATION '<hdfs_location>';
     
     
     CREATE EXTERNAL TABLE psn(
         id INT, 
         name STRING,
         habit ARRAY<STRING>,
         address MAP<STRING,STRING>
     )
     ROW FORMAT DELIMITED 
         FIELDS TERMINATED BY ','
         COLLECTION ITEMS TERMINATED BY '-'
         MAP KEYS TERMINATED BY ':'  
         LINES TERMINATED BY '\n'
     LOCATION '/usr';

#### 加载文件    
    LOAD DATA [LOCAL] INPATH '/data/mydata'  INTO TABLE psn;

## 4.分区

###                       a.创建分区
 #### 创建一个分区
     create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        PARTITIONED BY (age int)
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
 #### 创建两个个分区
      create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        PARTITIONED BY (age INT,sex STRING)
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
        LINES TERMINATED BY '\n';
#### 加载文件    
    LOAD DATA [LOCAL] INPATH '/data/mydata'  INTO TABLE psn PARTITION (age=20, sex='M');
#### 添加分区<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-AddPartitions'>help</a>:注意这个不是添加分区字段,是添加字段分区值
    ALTER TABLE psn add PARTITION (age=10,sex='WM')
#### 删除分区<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-DropPartitions'>help</a>
    ALTER TABLE psn DROP PARTITION (sex='WM')
#### 问题:外部表drop后会不会删除数据?
    CREATE EXTERNAL TABLE psn(
         id INT, 
         name STRING,
         habit ARRAY<STRING>,
         address MAP<STRING,STRING>
     )
     PARTITIONED BY (age int,sex string)
     ROW FORMAT DELIMITED 
         FIELDS TERMINATED BY ','
         COLLECTION ITEMS TERMINATED BY '-'
         MAP KEYS TERMINATED BY ':'  
         LINES TERMINATED BY '\n';

## 5.从其他表导入数据
    FROM psn
        INSERT OVERWRITE TABLE psn1  
        SELECT id,name,habit
        INSERT OVERWRITE TABLE psn2   
        SELECT id,name


    CREATE TABLE psn1(
         id INT, 
         name STRING,
         habit ARRAY<STRING>
     )
     ROW FORMAT DELIMITED 
         FIELDS TERMINATED BY ','
         COLLECTION ITEMS TERMINATED BY '-' 
         LINES TERMINATED BY '\n';
         
    CREATE TABLE psn2(
         id INT, 
         name STRING
     );

## 6.Hive Beeline(新的客户端) 
    需要配合server2服务使用,不用hive --service metastore &
    1.在node2上面,启动服务 hiveserver2
    2.beeline -u jdbc:hive2://node2:10000/default -n root
    
    命令以!开头
    (2)、beeline
    beeline> !connect jdbc:hive2://<host>:<port>/<db>;auth=noSasl root 123
    
    同时修改hadoop 配置文件 etc/hadoop/core-site.xml,加入如下配置项
    <property>
    <name>hadoop.proxyuser.root.hosts</name>
    <value>*</value>
    </property>
    <property>
        <name>hadoop.proxyuser.root.groups</name>
        <value>*</value>
    </property>

## 7.作业
   ### 1、实现struct例子:
        1,xiaoming:22
        2.xiaohong:33
        
        CREATE table student(
            id string,
            info struct<name:string,age:int>
        )
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ','
            COLLECTION ITEMS TERMINATED BY ':';
   ### 2、基站掉话率：找出掉线率最高的前10基站
       原表:
       create table call_monitor(
            record_time string,
            imei string,
            cell string,
            ph_num string,
            call_num string,
            drop_num double,
            duration double,
            drop_rate double,
            net_type string,
            erl string
       )
       ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',';
       加载数据
       load data local inpath '/root/cdr_summ_imei_cell_info.csv' into table call_monitor; 
       
       输出表:
       create table call_result(
            imei string,
            drop_rate double
       )
       ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',';
       #插入输出
       from call_monitor
       insert into call_result
           select imei,sum(drop_num)/sum(duration) drate group by imei order by drate desc
   ### 3、使用hive实现wordcount
       create table wordcount(
           line ARRAY<STRING>
       )
       ROW FORMAT DELIMITED 
       COLLECTION ITEMS TERMINATED BY ' ';
       
       load data local inpath '/root/data/wc' into table wordcount;
       
       select word,count(word) from(select explode(line)  as word from wordcount) a group by a.word;

### 8.参数调节
    1.在hive命令行,s输入set; 查看所以参数设置
    2.set hive.cli.print.header=true设置参数.
    3.编辑home目录下的.hiverc文件,加入默认设置

### 9.动态分区
   #### a)创建原始表:
     create table people_orgin(
            id int,
            name string,
            age int,
            sex string,
            habit array<string>,
            address map<string,string>
        ) 
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
       //加载数据
       load data local inpath '/root/people' into table people_orgin;
   #### b)创建分区表:
        create table people_partition(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        PARTITIONED BY (age int,sex string)
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
       //从原始表 加载数据
       from people_orgin
           insert overwrited people_partition select id,name,habit,address,age,sex distribute by age, sex;
       //
       from people_orgin
           insert overwrite table people_partition select id,name,habit,address,age,sex distribute by age, sex;
   ###  c)开启支持动态分区
    set hive.exec.dynamic.partition=true;
    默认：false
        set hive.exec.dynamic.partition.mode=nostrict;
    默认：strict（至少有一个分区列是静态分区）
    相关参数
        set hive.exec.max.dynamic.partitions.pernode;
    每一个执行mr节点上，允许创建的动态分区的最大数量(1000)
    set hive.exec.max.dynamic.partitions;
    所有执行mr节点上，允许创建的所有动态分区的最大数量(1000)
    set hive.exec.max.created.files;
    所有的mr job允许创建的文件的最大数量(100000)

### 10.hive 分桶
    分桶表是对列值取哈希值的方式，将不同数据放到不同文件中存储。
    适用场景：
        数据抽样（ sampling ）、map-join
    1.开启分通:set hive.enforce.bucketing=true;//hive3.1不用设置?
    2.创建分通表:
     create table people_bucket(
        id int,
        name string,
        age int,
        sex string,
        habit array<string>,
        address map<string,string>
    )
    CLUSTERED BY (id) INTO 4 BUCKETS;
    3.导入数据:
        from people_orgin 
            insert overwrite table people_bucket select id,name,age,sex,habit,address; 
    4.抽样
        select id, name, age from people_bucket tablesample(bucket 2 out of 4 on id);
        从第二个bucket开始,每隔4个bucket,获得一个文件,//2这里有4个文件,所以取出2号文件,也就是id %4==1的